# Default Preprocessing

## Loading Data

In [49]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

from IPython.display import display, HTML

In [50]:
path = os.getcwd()
path

'/Users/vishalbalaji/GitHub-repos/AI---1/Project'

In [51]:
# df_19 = pd.read_csv('players_19.csv')
# df_20 = pd.read_csv('players_20.csv')

df_19 = pd.read_csv(path + '/fifadataset/players_19.csv')
df_20 = pd.read_csv(path + '/fifadataset/players_20.csv')
df_19.head()

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,33,1985-02-05,187,83,Portugal,Juventus,...,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3
1,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,31,1987-06-24,170,72,Argentina,FC Barcelona,...,63+2,61+2,61+2,61+2,63+2,58+2,47+2,47+2,47+2,58+2
2,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Junior,26,1992-02-05,175,68,Brazil,Paris Saint-Germain,...,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3
3,193080,https://sofifa.com/player/193080/david-de-gea-...,De Gea,David De Gea Quintana,27,1990-11-07,193,76,Spain,Manchester United,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,27,1991-06-28,181,70,Belgium,Manchester City,...,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3


In [52]:
def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )

def tbl_report(tbl, cols=None, card=10):
    print("Table Shape", tbl.shape)
    dtypes = tbl.dtypes
    nulls = []
    uniques = []
    numuniques = []
    vcs = []
    for col in dtypes.index:
        n = tbl[col].isnull().sum()
        nulls.append(n)
        strdtcol = str(dtypes[col])
        #if strdtcol == 'object' or strdtcol[0:3] == 'int' or strdtcol[0:3] == 'int':
        #print(strdtcol)
        uniqs = tbl[col].unique()
        uniquenums = uniqs.shape[0]
        if uniquenums < card: # low cardinality
            valcounts = pd.value_counts(tbl[col], dropna=False)
            vc = "\n".join(["{}:{}".format(k,v) for k, v in valcounts.items()])
        else:
            vc='HC' # high cardinality
        uniques.append(uniqs)
        numuniques.append(uniquenums)
        vcs.append(vc)
    nullseries = pd.Series(nulls, index=dtypes.index)
    uniqueseries = pd.Series(uniques, index=dtypes.index)
    numuniqueseries = pd.Series(numuniques, index=dtypes.index)
    vcseries = pd.Series(vcs, index=dtypes.index)
    df = pd.concat([dtypes, nullseries, uniqueseries, numuniqueseries, vcseries], axis=1)
    df.columns = ['dtype', 'nulls', 'uniques', 'num_uniques', 'value_counts']
    if cols:
        return df[cols]
    return df

In [53]:
pretty_print(tbl_report(df_19, cols=['dtype', 'nulls', 'num_uniques', 'value_counts'], card = 11))

Table Shape (17770, 104)


,dtype,nulls,num_uniques,value_counts
sofifa_id,int64,0,17770,HC
player_url,object,0,17770,HC
short_name,object,0,16811,HC
long_name,object,0,17710,HC
age,int64,0,29,HC
dob,object,0,6093,HC
height_cm,int64,0,51,HC
weight_kg,int64,0,57,HC
nationality,object,0,160,HC
club,object,0,679,HC


## Column Transformations

### OneHot Encoding

In [54]:
def add_subtract(x):
    if '-' in x:
        return(int(x.split('-')[0]) - int(x.split('-')[1]))
    if '+' in x:
        return(int(x.split('+')[0]) + int(x.split('+')[1]))
    else:
        return(int(x))

In [55]:
change_cols = df_19.columns[list(df_19.columns).index('attacking_crossing'):]
for i in change_cols:
    df_19[i] = df_19[i].fillna('0').apply(lambda x: add_subtract(x))

In [56]:
def oh_normal(column):
    oh = OneHotEncoder()
    return(pd.DataFrame(oh.fit_transform(column).toarray(),columns=oh.categories_[0]).astype(int))
    
    


def comma(column):
    column = column.fillna('0')
    column = column.apply(lambda x: x.replace(' ',''))
    column = column.apply(lambda x: x.split(','))

    flat_list = [item for sublist in column for item in sublist]
    uniques = np.unique(np.asarray(flat_list))

    overall = []
    for i in column:
        zeroes = np.zeros(len(uniques)).astype(int)
        long = len(i)

        for j in range(long):
            zeroes[np.where(uniques == i[j])] = 1
        overall.append(zeroes)

    frame = pd.DataFrame(overall,columns=uniques)

    return(frame)

In [57]:
# One hot encode: Nationality, Club, Player Positions, Preferred Foot, Body Type, Player Tags, Team Position, Nation Position, Player Traits
oh_cols = ['nationality','club','team_position']
oh_comma = ['player_positions','player_traits'] 

In [58]:
#One hot encoding for columns with comma separated values
pos_df = comma(df_19[oh_comma[0]])
df_19 = df_19.drop(oh_comma[0],axis=1).join(pos_df)

traits_df = comma(df_19[oh_comma[1]]).drop('0',axis = 1)
df_19 = df_19.drop(oh_comma[1],axis=1).join(traits_df)

In [59]:
#One hot encoding for normal columns
nationality_df = oh_normal(df_19[[oh_cols[0]]])
df_19 = df_19.drop(oh_cols[0],axis=1).join(nationality_df)

club_df = oh_normal(df_19[[oh_cols[1]]])
df_19 = df_19.drop(oh_cols[1],axis=1).join(club_df, rsuffix='club_')

team_pos_df = oh_normal(df_19[[oh_cols[2]]].fillna('0')).iloc[:, :-1]
df_19 = df_19.drop(oh_cols[2],axis=1).join(team_pos_df,rsuffix='team_')

### Drop columns

In [60]:
cols_to_drop = ['player_url','short_name','long_name','dob','potential','real_face','player_tags','loaned_from','nation_position','nation_jersey_number','team_jersey_number']
df_19 = df_19.drop(cols_to_drop,axis=1)

In [61]:
df_19.loc[df_19['preferred_foot'] == 'Right', 'preferred_foot'] = 1
df_19.loc[df_19['preferred_foot'] == 'Left', 'preferred_foot'] = 0

In [62]:
attribute_cols = df_19.columns[list(df_19.columns).index('pace'):list(df_19.columns).index('attacking_crossing')]
df_19[attribute_cols] = df_19[attribute_cols].fillna(value=0).astype(int)
df_19[['release_clause_eur']] = df_19[['release_clause_eur']].fillna(value=0).astype(int)

In [159]:
# pretty_print(tbl_report(df_19, cols=['dtype', 'nulls', 'num_uniques', 'value_counts'], card = 11))

Pre-process column join and contract_valid_till

### Feature Transformations

In [64]:
df_19['joined'].isnull().sum()

1504

In [65]:
df_19['joined'] =pd.to_datetime(df_19['joined'])

In [66]:
df_19['joined_year'] = pd.DatetimeIndex(df_19['joined']).year

In [67]:
df_19['joined_year'] = df_19['joined_year'].replace(np.nan,df_19['joined_year'].median())

In [68]:
df_19['contract_valid_until'] = df_19['contract_valid_until'].replace(np.nan,df_19['contract_valid_until'].median())

In [69]:
df_19['total_no_years'] = df_19['contract_valid_until']-df_19['joined_year']

In [70]:
df_19.drop(['contract_valid_until','joined_year','joined'],axis=1,inplace=True)

In [71]:
df_19.shape

(17770, 1007)

work rate

In [72]:
work_rate = df_19['work_rate']

In [73]:
df_work = oh_normal(work_rate.values.reshape(-1,1))
df_work.shape

(17770, 9)

In [74]:
df_19.shape

(17770, 1007)

In [75]:
df_19 = pd.concat([df_19,df_work],axis=1)
df_19.shape

(17770, 1016)

body_type

In [76]:
df_19.head()

,sofifa_id,age,height_cm,weight_kg,overall,value_eur,wage_eur,preferred_foot,international_reputation,weak_foot,...,total_no_years,High/High,High/Low,High/Medium,Low/High,Low/Low,Low/Medium,Medium/High,Medium/Low,Medium/Medium
0,20801,33,187,83,94,77000000,405000,1,5,4,...,4.0,0,1,0,0,0,0,0,0,0
1,158023,31,170,72,94,110500000,565000,0,5,4,...,17.0,0,0,0,0,0,0,0,0,1
2,190871,26,175,68,92,118500000,290000,1,5,5,...,5.0,0,0,1,0,0,0,0,0,0
3,193080,27,193,76,91,72000000,260000,1,4,3,...,8.0,0,0,0,0,0,0,0,0,1
4,192985,27,181,70,91,102000000,355000,1,4,5,...,8.0,1,0,0,0,0,0,0,0,0


In [77]:
df_19['body_type'].value_counts()

Normal                 10410
Lean                    6268
Stocky                  1085
C. Ronaldo                 1
Messi                      1
Neymar                     1
Courtois                   1
PLAYER_BODY_TYPE_25        1
Shaqiri                    1
Akinfenwa                  1
Name: body_type, dtype: int64

In [78]:
df_19['body_type'].unique()

array(['C. Ronaldo', 'Messi', 'Neymar', 'Lean', 'Normal', 'Courtois',
       'Stocky', 'PLAYER_BODY_TYPE_25', 'Shaqiri', 'Akinfenwa'],
      dtype=object)

In [79]:
for name in ['C. Ronaldo', 'Messi', 'Neymar', 'Courtois',
        'PLAYER_BODY_TYPE_25', 'Shaqiri', 'Akinfenwa']:
    idx = df_19[df_19['body_type'] == name].index.values
    df_19.loc[idx,'body_type'] = 'Normal'

min_max_scale - player_traits

In [80]:
df_19.head(2)

,sofifa_id,age,height_cm,weight_kg,overall,value_eur,wage_eur,preferred_foot,international_reputation,weak_foot,...,total_no_years,High/High,High/Low,High/Medium,Low/High,Low/Low,Low/Medium,Medium/High,Medium/Low,Medium/Medium
0,20801,33,187,83,94,77000000,405000,1,5,4,...,4.0,0,1,0,0,0,0,0,0,0
1,158023,31,170,72,94,110500000,565000,0,5,4,...,17.0,0,0,0,0,0,0,0,0,1


preprocessin - min_max scale player skills

In [81]:
player_skills = ['pace', 'shooting','passing','dribbling','defending','physic','gk_diving','gk_handling','gk_kicking','gk_reflexes','gk_speed','gk_positioning']


In [82]:
for col in player_skills:
    df_19[col]=(df_19[col]-df_19[col].min())/(df_19[col].max()-df_19[col].min())

Standardize scale wage and value

In [83]:
wage_n_value = ['wage_eur','value_eur']

In [84]:
for col in wage_n_value:
    df_19[col]=(df_19[col]-df_19[col].min())/(df_19[col].max()-df_19[col].min())

In [85]:
df_19[wage_n_value].corr()

,wage_eur,value_eur
wage_eur,1.000000,0.861424
value_eur,0.861424,1.000000


In [86]:
df_19.drop(['wage_eur'],axis=1,inplace=True)

update prefered foot int64

In [87]:
df_19['preferred_foot'] = df_19['preferred_foot'].astype(int)

# Part-B Specific Preprocessing

## Loading Data

In [110]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from IPython.display import display, HTML

In [89]:
path = os.getcwd()
path

'/Users/vishalbalaji/GitHub-repos/AI---1/Project'

In [90]:
# df_19 = pd.read_csv('players_19.csv')
# df_20 = pd.read_csv('players_20.csv')

df_19 = pd.read_csv(path + '/fifadataset/players_19.csv')
df_20 = pd.read_csv(path + '/fifadataset/players_20.csv')
df_19.head()

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,33,1985-02-05,187,83,Portugal,Juventus,...,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3
1,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,31,1987-06-24,170,72,Argentina,FC Barcelona,...,63+2,61+2,61+2,61+2,63+2,58+2,47+2,47+2,47+2,58+2
2,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Junior,26,1992-02-05,175,68,Brazil,Paris Saint-Germain,...,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3
3,193080,https://sofifa.com/player/193080/david-de-gea-...,De Gea,David De Gea Quintana,27,1990-11-07,193,76,Spain,Manchester United,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,27,1991-06-28,181,70,Belgium,Manchester City,...,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3


In [91]:
def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )

def tbl_report(tbl, cols=None, card=10):
    print("Table Shape", tbl.shape)
    dtypes = tbl.dtypes
    nulls = []
    uniques = []
    numuniques = []
    vcs = []
    for col in dtypes.index:
        n = tbl[col].isnull().sum()
        nulls.append(n)
        strdtcol = str(dtypes[col])
        #if strdtcol == 'object' or strdtcol[0:3] == 'int' or strdtcol[0:3] == 'int':
        #print(strdtcol)
        uniqs = tbl[col].unique()
        uniquenums = uniqs.shape[0]
        if uniquenums < card: # low cardinality
            valcounts = pd.value_counts(tbl[col], dropna=False)
            vc = "\n".join(["{}:{}".format(k,v) for k, v in valcounts.items()])
        else:
            vc='HC' # high cardinality
        uniques.append(uniqs)
        numuniques.append(uniquenums)
        vcs.append(vc)
    nullseries = pd.Series(nulls, index=dtypes.index)
    uniqueseries = pd.Series(uniques, index=dtypes.index)
    numuniqueseries = pd.Series(numuniques, index=dtypes.index)
    vcseries = pd.Series(vcs, index=dtypes.index)
    df = pd.concat([dtypes, nullseries, uniqueseries, numuniqueseries, vcseries], axis=1)
    df.columns = ['dtype', 'nulls', 'uniques', 'num_uniques', 'value_counts']
    if cols:
        return df[cols]
    return df

In [92]:
pretty_print(tbl_report(df_19, cols=['dtype', 'nulls', 'num_uniques', 'value_counts'], card = 11))

Table Shape (17770, 104)


,dtype,nulls,num_uniques,value_counts
sofifa_id,int64,0,17770,HC
player_url,object,0,17770,HC
short_name,object,0,16811,HC
long_name,object,0,17710,HC
age,int64,0,29,HC
dob,object,0,6093,HC
height_cm,int64,0,51,HC
weight_kg,int64,0,57,HC
nationality,object,0,160,HC
club,object,0,679,HC


## Column Transformations

### OneHot Encoding

In [93]:
def add_subtract(x):
    if '-' in x:
        return(int(x.split('-')[0]) - int(x.split('-')[1]))
    if '+' in x:
        return(int(x.split('+')[0]) + int(x.split('+')[1]))
    else:
        return(int(x))

In [94]:
change_cols = df_19.columns[list(df_19.columns).index('attacking_crossing'):]
for i in change_cols:
    df_19[i] = df_19[i].fillna('0').apply(lambda x: add_subtract(x))

In [95]:
def oh_normal(column):
    oh = OneHotEncoder()
    return(pd.DataFrame(oh.fit_transform(column).toarray(),columns=oh.categories_[0]).astype(int))
    
    


def comma(column):
    column = column.fillna('0')
    column = column.apply(lambda x: x.replace(' ',''))
    column = column.apply(lambda x: x.split(','))

    flat_list = [item for sublist in column for item in sublist]
    uniques = np.unique(np.asarray(flat_list))

    overall = []
    for i in column:
        zeroes = np.zeros(len(uniques)).astype(int)
        long = len(i)

        for j in range(long):
            zeroes[np.where(uniques == i[j])] = 1
        overall.append(zeroes)

    frame = pd.DataFrame(overall,columns=uniques)

    return(frame)

In [96]:
oh_cols = ['nationality','club','team_position']
oh_comma = ['player_traits'] 

In [97]:
#One hot encoding for columns with comma separated values
traits_df = comma(df_19[oh_comma[0]]).drop('0',axis = 1)
df_19 = df_19.drop(oh_comma[0],axis=1).join(traits_df)

In [98]:
pretty_print(tbl_report(df_19, cols=['dtype', 'nulls', 'num_uniques', 'value_counts'], card = 11))

Table Shape (17770, 140)


,dtype,nulls,num_uniques,value_counts
sofifa_id,int64,0,17770,HC
player_url,object,0,17770,HC
short_name,object,0,16811,HC
long_name,object,0,17710,HC
age,int64,0,29,HC
dob,object,0,6093,HC
height_cm,int64,0,51,HC
weight_kg,int64,0,57,HC
nationality,object,0,160,HC
club,object,0,679,HC


### Getting Predictor and Response Variables

#### Predictor Variables

In [104]:
attribute_cols = df_19.columns[list(df_19.columns).index('pace'):list(df_19.columns).index('ls')]
# player_pos = df_19.columns[[list(df_19.columns).index('player_positions')]]
# attribute_cols = attribute_cols.append(player_pos)
attribute_cols

Index(['pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_reflexes', 'gk_speed',
       'gk_positioning', 'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power',
       'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
       'mentality_aggression', 'mentality_interceptions',
       'mentality_positioning', 'mentality_vision', 'mentality_penalties',
       'mentality_composure', 'defending_marking', 'defending_standing_tackle',
       'defending_sliding_tackle', 'goalkeeping_diving',
       'goalkeeping_handling', 'goalkeeping_kicking',
       'goalkeeping_positioning',

In [105]:
traits_cols = df_19.columns[list(df_19.columns).index('1-on-1Rush'):]
sum_cols = attribute_cols.append(traits_cols)
# player_pos = df_19.columns[[list(df_19.columns).index('player_positions')]]
# sum_cols = sum_cols.append(player_pos)
sum_cols

Index(['pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_reflexes', 'gk_speed',
       'gk_positioning', 'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power',
       'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
       'mentality_aggression', 'mentality_interceptions',
       'mentality_positioning', 'mentality_vision', 'mentality_penalties',
       'mentality_composure', 'defending_marking', 'defending_standing_tackle',
       'defending_sliding_tackle', 'goalkeeping_diving',
       'goalkeeping_handling', 'goalkeeping_kicking',
       'goalkeeping_positioning',

In [109]:
df_attributes = df_19[attribute_cols].fillna(0).astype(int)
df_total = df_19[sum_cols].fillna(0).astype(int)

Final training dataframe with only player attributes

In [113]:
minmax = MinMaxScaler()
attributes_train = pd.DataFrame(minmax.fit_transform(df_attributes),columns=df_attributes.columns)

Final training dataframe with player attributes and player traits

In [115]:
minmax = MinMaxScaler()
total_train = pd.DataFrame(minmax.fit_transform(df_total),columns=df_total.columns)

#### Response variables

In [145]:
player_pos_df = df_19['player_positions']

Random selection of variables from each row

In [149]:
random_pos = []
for i in player_pos_df:
    dummy = i.split(',')
    random_pos.append(dummy[np.random.randint(0,len(dummy))].strip())

In [173]:
positions = pd.Series(random_pos).unique()
positions.sort()

In [174]:
positions

array(['CAM', 'CB', 'CDM', 'CF', 'CM', 'GK', 'LB', 'LM', 'LW', 'LWB',
       'RB', 'RM', 'RW', 'RWB', 'ST'], dtype=object)

Bucketing in GK, Defence, Midfield, Offence

In [161]:
mapping_dict = {
'GK':0 ,
'LWB':1 ,
'LB':1 ,
'LCB':1 ,
'CB':1 ,
'RCB':1 ,
'RB':1 ,
'RWB':1 ,
'LDM':2 ,
'CDM':2 ,
'RDM':2 ,
'LM':2 ,
'LCM':2 ,
'CM':2 ,
'RCM':2 ,
'RM':2 ,
'LAM':2 ,
'CAM':2 ,
'RAM':2 ,
'LW':3 ,
'LF':3 ,
'CF':3 ,
'RF':3 ,
'RW':3 ,
'LS':3 ,
'ST':3 ,
'RS':3 ,
}

In [167]:
mapped_pos_buckets = []
for i in random_pos:
    mapped_pos_buckets.append(mapping_dict[i])

Bucketing in full

In [175]:
mapping_dict_full = {}
for i in range(len(positions)):
    mapping_dict_full[positions[i]] = i

In [176]:
mapping_dict_full

{'CAM': 0,
 'CB': 1,
 'CDM': 2,
 'CF': 3,
 'CM': 4,
 'GK': 5,
 'LB': 6,
 'LM': 7,
 'LW': 8,
 'LWB': 9,
 'RB': 10,
 'RM': 11,
 'RW': 12,
 'RWB': 13,
 'ST': 14}

In [177]:
mapped_pos_full = []
for i in random_pos:
    mapped_pos_full.append(mapping_dict_full[i])